This analyses the attention layer of the following network:

input -> gru -> attention -> classification

In [1]:
cd ../../../src

[Errno 2] No such file or directory: '../../../src'
/Users/cock/kDrive/PhD/Projects/labs/behavioral-bias-investigation/notebooks/results plotter


Attention layer analysis from the "only attention" algorithm. 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import pickle
import numpy as np
from utils.statistical import fischer_mean

import keras
import tensorflow as tf
from ml.models.classifiers.rnn_attention import RNNAttentionModel
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'utils'

# Data

In [4]:
path = 'path/nonnested_xval_mltsattpt_lfull_modelseeds940.pkl'
with open(path, 'rb') as fp:
    res = pickle.load(fp)
    
    
# sim dict pm
sim_pm = 'path_to_demographics'
with open(sim_pm, 'rb') as fp:
    simpm = pickle.load(fp)


In [5]:
exp_path = 'path/mrnn_attention_f10/2024_01_25_0/'
experiment = []
for (dirpath, _, filenames) in os.walk(exp_path):
    files = [os.path.join(dirpath, file) for file in filenames]
    experiment.extend(files)
checkpoints = [cfile for cfile in experiment if cfile.endswith('model.pth')]
checkpoints = np.unique(checkpoints)
print('There are {} files'.format(len(checkpoints)))

config = [exp for exp in experiment if 'config' in exp][0]
with open(config, 'rb') as fp:
    config = pickle.load(fp)
    config['ml']['models']['maxlen'] = 619
    config['n_classes'] = 2
    config['experiment']['nclasses'] = 2


There are 10 files


In [7]:
# Return Attention weights
def get_attention(model, x):
    x_lengths = [len(xxx) for xxx in x]
    xx, _ = model._format_features(x)
    attention_weights = model.get_attention_outputs(xx)
    assert len(attention_weights) == len(x_lengths)
    return attention_weights, x_lengths


In [8]:
# Maps
attribute_to_shortcut = {
    'attribute': {
        'attb': 'minority',
        'attn': 'minority',
        'attn': 'attn'
        
    },
    'vector_binary': {
        '000': 'label0',
        '001': 'label0',
        '010': 'label0',
        '100': 'label0',
        '011': 'label1',
        '101': 'label1',
        '110': 'label1',
        '111': 'label1'
    },
    'gender': {
        1: 'male',
        2: 'female',
        3: 'genderminority',
        4: 'genderminority'
    },
    'sa_gru':{
        'sagru0': 'sagru0',
        'sagru1': 'sagru1'
    }
}
    

In [9]:
pre_path = '/Users/cock/kDrive/PhD/Projects/labs/behavioural-pattern-mining/experiments/pattern-mining/EDM24/beerslaw/gru_attention/datachemlab_beerslaw/mrnn_attention_f10/2024_01_25_0/'
post_path = '/logger/ltsattpt/ctgru_nlayers1_ncells16_bs16_ep50/_model.pth'
models = {
    'all_ids': [],
    'all_gender': [],
    'all_gender_truth': [],
    'all_lengths': [],
    'all_sequences': [],
    'all_truths' :[],
    'all_attention_weights': [],
    'all_predictions': [],
    'all_prediction_truth': [],
    'all_gender_prediction_truth': [],
}
for fold in range(10):
    model_path = '{}{}{}'.format(pre_path, fold, post_path)
    model = RNNAttentionModel(config)
    model.load_model_weights(res['x'], model_path)
    fold_input = [res['x'][idx] for idx in res[fold]['test_index']]
    attention_weights, attention_lengths = get_attention(model, fold_input)
    
    models[fold] = {
        'path': '{}{}{}'.format(pre_path, fold, post_path), 
        'model': model,
        'input': fold_input,
        'weights': attention_weights,
        'gender': [simpm['sequences'][idx]['gender'] for idx in res[fold]['test_index']],
        'language': [simpm['sequences'][idx]['language'].lower().replace('ç', 'c') for idx in res[fold]['test_index']]
    }

    models['all_ids'] = [*models['all_ids'], *[idx for idx in res[fold]['test_index']]]
    models['all_lengths'] = [*models['all_lengths'], *attention_lengths]
    models['all_sequences'] = [*models['all_sequences'], *fold_input]
    models['all_attention_weights'] = [*models['all_attention_weights'], *attention_weights]
    models['all_gender'] = [*
                            models['all_gender'], 
                            *[attribute_to_shortcut['gender'][simpm['sequences'][idx]['gender']] for idx in res[fold]['test_index']]
    ]
    models['all_truths'] = [*models['all_truths'], *[attribute_to_shortcut['vector_binary'][simpm['sequences'][idx]['vector_binary']] for idx in res[fold]['test_index']]]
    models['all_predictions'] = [*models['all_predictions'], *[attribute_to_shortcut['sa_gru'][simpm['sequences'][idx]['sa_gru']] for idx in res[fold]['test_index']]]
    models['all_gender_truth'] = [
        *models['all_gender_truth'], 
        *[
            '{}_{}'.format(
                attribute_to_shortcut['gender'][simpm['sequences'][idx]['gender']] ,
                attribute_to_shortcut['vector_binary'][simpm['sequences'][idx]['vector_binary']]
            ) for idx in res[fold]['test_index']
        ]        
    ]
    models['all_prediction_truth'] = [
        *models['all_prediction_truth'], 
        *[
            '{}_{}'.format(
                attribute_to_shortcut['sa_gru'][simpm['sequences'][idx]['sa_gru']],
                attribute_to_shortcut['vector_binary'][simpm['sequences'][idx]['vector_binary']]
            ) for idx in res[fold]['test_index']
        ]        
    ]
    models['all_gender_prediction_truth'] = [
        *models['all_gender_prediction_truth'], 
        *[
            '{}_{}_{}'.format(
                attribute_to_shortcut['gender'][simpm['sequences'][idx]['gender']] ,
                attribute_to_shortcut['sa_gru'][simpm['sequences'][idx]['sa_gru']],
                attribute_to_shortcut['vector_binary'][simpm['sequences'][idx]['vector_binary']]
            ) for idx in res[fold]['test_index']
        ]        
    ]
    
    


# Attention Processing

# Five only one part of the feature

# Statistical Tests

In [10]:
weights = models

In [11]:
colours = [
    'orange', 'darkgoldenrod', 'darkkhaki', 'lightseagreen', 
    'steelblue', 'dodgerblue', 'lightsteelblue',
    'slateblue', 'thistle', 'orchid', 'deeppink'
]

features = [
    'greengreen',
    'greenred',
    'notgreennotred',
    'noobserved',
    'other',
    'concentration',
    'width',
    'pdf',
    'break',
    'end_sequence'
]

char_map = {
    'male': 'gender',
    'female': 'gender',
    'sagru0': 'prediction',
    'sagru1': 'prediction',
    'label0': 'truth',
    'label1': 'truth'
}

In [12]:
def get_indices_demographics(characteristics):
    # Get demographics
    characs = characteristics.split('.')
    characteristics_list = [char_map[char] for char in characs]
    char_sort = np.argsort(characteristics_list)
    characteristics_list = [characteristics_list[idx_sort] for idx_sort in char_sort]
    characteristics_list = '_'.join(characteristics_list)
    characteristics_string = 'all_{}'.format(characteristics_list)

    # attributes
    characs = [characs[idx_sort] for idx_sort in char_sort]
    attributes_string = '_'.join(characs)

    indices = [
        i for i in range(len(weights[characteristics_string]))
        if weights[characteristics_string][i] == attributes_string
    ]
    
    return indices

In [13]:
def statistical_test(weights_a, weights_b):
    return fischer_mean(weights_a, weights_b) 

def distribution_attention_weights(weights_a, weights_b):
    print('{} ATTENTION Weight Plots {}'.format('*' * 50, '*' * 50))
    cols = np.random.choice(colours, size=2, replace=False)
    n_features = len(weights_a[0])
    wsa, wsb = [], []
    for feature in range(n_features):
        for student in weights_a:
            wsa = [*wsa, *student[feature]]
        for student in weights_b:
            wsb = [*wsb, *student[feature]]

        plt.figure(figsize=(12, 4))
        plt.hist(wsa, bins=np.arange(0, 1.01, 0.01), color=cols[0], alpha=0.3, label='a', density=True)
        plt.hist(wsb, bins=np.arange(0, 1.01, 0.01), color=cols[1], alpha=0.3, label='b', density=True)
        p_value = statistical_test(wsa, wsb)
        plt.title('Distribution of activation values for features {}\n[p-value = {}]'.format(feature, p_value))
        plt.legend()
        plt.show()

def pvalue_attention_weights(weights_a, weights_b, p_value_significance=0.05):
    print('{} ATTENTION Weight Differences {}'.format('*' * 50, '*' * 50))
    cols = np.random.choice(colours, size=2, replace=False)
    plot_p_values = {}
    p_value_count = 0
    n_features = len(weights_a[0])
    wsa, wsb = [], []
    
    for feature in range(n_features):
        for student in weights_a:
            wsa.append(student[feature])
        for student in weights_b:
            wsb.append(student[feature])
            
        p_value = statistical_test(wsa, wsb)
        plot_p_values[feature] = p_value
        if p_value <= p_value_significance:
            print('difference between a and b with feature {} [{}]'.format(feature, p_value))
            p_value_count += 1
    print('Total difference: {}'.format(p_value_count))
    return p_value_count

def compare(char_a, char_b, plot_insteadof_print=False, pvalue_significant=0.05):
    print('Comparing students for group:')
    print('   group A: {}'.format(
        ' '.join(char_a.split('.'))
    ))
    print('   group B: {}'.format(
        ' '.join(char_b.split('.'))
    ))

    indices_a = get_indices_demographics(char_a)
    indices_b = get_indices_demographics(char_b)

    weights_a = [weights['all_attention_weights'][idxa] for idxa in indices_a]
    weights_b = [weights['all_attention_weights'][idxb] for idxb in indices_b]

    

    if plot_insteadof_print:
        distribution_attention_weights(weights_a, weights_b)
    else:
        pvalue_attention_weights(weights_a, weights_b, pvalue_significant)


## No correction

In [14]:
compare('female.label1', 'female.label0', pvalue_significant=0.05)

Comparing students for group:
   group A: female label1
   group B: female label0
************************************************** ATTENTION Weight Differences **************************************************
Total difference: 0


In [ ]:
compare('male.label1', 'male.label0', pvalue_significant=0.05)

Comparing students for group:
   group A: male label1
   group B: male label0
************************************************** ATTENTION Weight Differences **************************************************


In [ ]:
compare('female.label1', 'male.label1', pvalue_significant=0.05)

In [ ]:
compare('female.label0', 'male.label0', pvalue_significant=0.05)